In [ ]:
# import libraries
from cil.optimisation.algorithms import SIRT, CGLS
from cil.framework import ImageGeometry, AcquisitionGeometry, AcquisitionData
from cil.plugins.astra.operators import ProjectionOperator
from cil.plugins.astra.processors import FBP
from cil.utilities.display import show2D

import numpy as np
import os, sys

import tomophantom
from tomophantom import TomoP2D

In [ ]:
# Load a tomophantom image 
model = 12 # select a model number from the library
N = 256 # set dimension of the phantom
path = os.path.dirname(tomophantom.__file__)
path_library2D = os.path.join(path, "Phantom2DLibrary.dat")

phantom2D_np = TomoP2D.Model(model, N, path_library2D)    
ig = ImageGeometry(voxel_num_x=N, voxel_num_y=N, voxel_size_x = 0.1, voxel_size_y = 0.1)

phantom2D = ig.allocate()
phantom2D.fill(phantom2D_np)

In [ ]:
# Create Acquisition Geometry
detectors =  N
angles = np.linspace(0, 180, 180, dtype=np.float32)

ag = AcquisitionGeometry.create_Parallel2D()\
                        .set_angles(angles)\
                        .set_panel(detectors, pixel_size=0.1)

In [ ]:
device = "gpu"

In [ ]:
# Create Projection Operator
A = ProjectionOperator(ig, ag, device )

In [ ]:
# Create projection data and corrupt with noise
np.random.seed(10)

# inverse crime
sino = A.direct(phantom2D)

# no inverse crime
# sino_analytic = TomoP2D.ModelSino(model, N, detectors, angles, path_library2D)

noisy_data = ag.allocate()
noisy_data.fill(sino + np.random.normal(0, 0.5, ag.shape))

In [ ]:
# Simple Backprojection
bp = A.adjoint(noisy_data)

In [ ]:
# FBP reconstruction
fbp = FBP(ig, ag)(noisy_data)

In [ ]:
# Setup and run SIRT reconstruction
sirt = SIRT(initial=ig.allocate(), operator = A, data = noisy_data, 
            max_iteration=100, update_objective_interval=10)
sirt.run(100, verbose=1)

In [ ]:
# Setup and run CGLS reconstruction
cgls = CGLS(initial=ig.allocate(), operator = A, data = noisy_data, 
            max_iteration=100, update_objective_interval=5)
cgls.run(20, verbose=1)

In [ ]:
show2D([phantom2D, noisy_data, bp, fbp, sirt.solution, cgls.solution],
          title=["Phantom", "Noisy Projection Data", "Back Projection", "Filtered BackProjection", "SIRT", "CGLS"],
         origin = "upper", cmap="inferno")